In [18]:
import os
import re
import pandas as pd
from collections import namedtuple

In [19]:
home = os.path.expanduser('~')

In [20]:
folder_ner = f"{home}/nlp/texts/ner/"

In [21]:
folder_snd = f"{home}/nlp/texts/snd/"

In [22]:
exe_snd = os.path.join(folder_snd, '0597IbnJawzi.Mawducat.ShamAY0034138_3_snd.tsv')

In [23]:
exe_ner = os.path.join(folder_ner, '0597IbnJawzi.Mawducat.ShamAY0034138_3_ner.tsv')

In [2]:
folder = f"{os.path.expanduser('~')}/nlu/project/texts/snd/"

In [3]:
muwatta = f"{os.path.expanduser('~')}/nlu/project/texts/snd/muwatta/"

In [4]:
muwatta_conll = f"{os.path.expanduser('~')}/nlu/project/texts/snd/muwatta_conll/"

In [5]:
ibnjawzi = f"{os.path.expanduser('~')}/nlu/project/texts/snd/ibnjawzi/"

In [6]:
ibnjawzi_conll = f"{os.path.expanduser('~')}/nlu/project/texts/snd/ibnjawzi_conll/"

In [7]:
file_tsv = os.path.join(folder,
                   '0179MalikIbnAnas.Muwatta.IbnZayd.ShamAY0033922_manual-SND.tsv')

In [8]:
file_conll = os.path.join(folder,
                   '0179MalikIbnAnas.Muwatta.IbnZayd.ShamAY0033922_manual-SND.conll')

In [9]:
manual_snd = f"{os.path.expanduser('~')}/nlu/project/texts/snd/manual_snd/"

In [24]:
def df_to_conll(df, filename, directory):
    df.to_csv(os.path.join(directory, filename),
              sep='\t', header=False, index=False)

In [25]:
def clean_conll(path_to_file, separator=' '):
    
    df = pd.read_csv(path_to_file, sep=' ', header=None)
    df.columns = ['tokens', 'labels']
    
    for idx, row in df.iterrows():
        if df.at[idx, 'labels'] == 'S':
            if df.at[idx-1, 'labels'] == '_':
                df.at[idx, 'labels'] = 'B'
            elif df.at[idx+1, 'labels'] == '_':
                 df.at[idx, 'labels'] = 'E'
            else:
                df.at[idx, 'labels'] = 'I'
                
    arabicRegex = "[ذ١٢٣٤٥٦٧٨٩٠ّـضصثقفغعهخحجدًٌَُلإإشسيبلاتنمكطٍِلأأـئءؤرلاىةوزظْلآآ]+|#"
    
    df.loc[df.tokens.str.contains(arabicRegex, regex = True)]
    
    df.loc[df.tokens.str.match('#'), 'labels'] = '#'
    
    for idx, row in df.iterrows():
        if df.at[idx, 'labels'] == '#' and df.at[idx+1, 'labels'] == '#':
            df.drop(idx, inplace=True)
            
    df = df.replace(['#'], separator)
    
    return df
    

In [10]:
for file in os.listdir(folder):
    if file.endswith('.conll'):
        dataframe = clean_conll(os.path.join(folder, file))
        
        df_to_conll(dataframe, file, manual_snd)

NameError: name 'folder' is not defined

In [26]:
def webanno_blockwise(wa:str) -> list:
    return wa.split('\n\n')

In [27]:
def grouper(input_list, n=2):
    for i in range(len(input_list) - (n - 1)):
        yield input_list[i:i+n]

In [32]:
AnnoLine = namedtuple("AnnoLine", "sent_tok_idx doc_chr_idx token label empty")

def wablock_to_conll(wab:str, label_mapper) -> str:
    """Return conll block from wab block, map labels via label_mapper.
    """
    
    arabic_re = re.compile("[ذ١٢٣٤٥٦٧٨٩٠ّـضصثقفغعهخحجدًٌَُلإإشسيبلاتنمكطٍِلأأـئءؤرلاىةوزظْلآآ]+")
    
    is_content = lambda l: l and not l.startswith('#')
    is_arabic = lambda l: arabic_re.search(l) is not None
    
    # we pad blocks because the label_mapper processes only interior lines
    # i.e. it drops first and last line of each block
    pad_str = '_\t_\t_\t_\t_'
    pad_block = lambda b: [pad_str] + b + [pad_str]  
    
    content_tups = (AnnoLine(*l.split('\t')) 
                    for l in pad_block(wab.splitlines()) if is_content(l) and (is_arabic(l) or l==pad_str))
  
    # process each line in context
    return '\n'.join([' '.join([cur.token, label_mapper(prv.label, cur.label, nxt.label)]) \
                      for prv, cur, nxt in grouper(list(content_tups), 3)])

In [29]:
def wa_to_BIEO_labels(prv:str, cur:str, nxt:str) -> str:
    """Returns one of B I E O based on the WebAnno labels."""
    
    # WebAnno labels tokens '_' or 'S[#]' where # is a digit. We only need the first char.
    key = prv[0]+cur[0]+nxt[0]
    
    return dict(SSS='I', _SS='B', SS_='E').get(key,'O')

In [30]:
def wa_to_BIO_labels(prv:str, cur:str, nxt:str) -> str:
    """Returns one of B I O based on the WebAnno labels."""
    
    # WebAnno labels tokens '_' or 'S[#]' where # is a digit. We only need the first char.
    key = prv[0]+cur[0]+nxt[0]
    
    return dict(SSS='I', _SS='B', SS_='I').get(key,'O')

In [173]:
with open(file_tsv, 'r') as f:
    
    print('\n\n'.join([wablock_to_conll(b, wa_to_BIEO_labels) for b in webanno_blockwise(f.read())]))
    



بسم O
الله O
الرحمن O
الرحيم O

وصلاته O
وسلامه O
على O
أشرف O
المرسلين O

الضحايا O

حدثنا B
علي I
بن I
زياد I
عن I
مالك I
بن I
أنس I
عن I
عمرو I
بن I
الحارث I
عن I
عبيد I
بن I
فيروز I
عن I
البر I
اء I
بن I
عازب E
أن O
رسول O
الله O
صلى O
الله O
عليه O
وسلم O
سئل O
ماذا O
يتقى O
من O
الضحايا O
فأشار O
بيده O
فقال O
أربعا O
وكان O
البراء O
يشير O
بيده O
ويقول O
ويدي O
أقصر O
من O
يد O
رسول O
الله O
صلى O
الله O
عليه O
وسلم O
العرجاء O
البين O
ضلعها O
والعوراء O
البين O
عورها O
والمريضة O
البين O
مرضها O
والعجفاء O
التي O
لا O
تنقي O

وحدثنا B
عن I
مالك I
عن I
نافع I
عن I
عبد I
الله I
بن I
عمر E
أنه O
كان O
يقول O
الضحايا O
والبدن O
الثني O
فما O
فوقه O

وحدثنا B
عن I
نافع I
عن I
ابن I
عمر E
أنه O
كان O
يقول O
يتقى O
من O
الضحايا O
والبدن O
التي O
لم O
تسن O
والتي O
نقص O
من O
خلقها O

وحدثنا B
عن I
نافع I
عن I
ابن I
عمر E
أنه O
ضحى O
مرة O
بالمدينة O
قال O
نافع O
فأمرني O
أن O
أشتري O
له O
كبشا O
فحيلا O
أقرن O
ثم O
أذبحه O
يوم O
الأضحى O
في O
مصلى O
الناس O
قال O
نافع O
ففعلت O
ثم O

In [33]:
with open(os.path.join(folder_ner, '0597IbnJawzi.Mawducat.ShamAY0034138_3_ner.tsv'), 'r') as f:
    
    text = '\n\n'.join([wablock_to_conll(b, wa_to_BIO_labels) for b in webanno_blockwise(f.read())])

TypeError: __new__() takes 6 positional arguments but 7 were given

In [71]:
with open(os.path.join(folder_ner, exe_snd), 'r') as f:
    
    text = '\n\n'.join([wablock_to_conll(b, wa_to_BIEO_labels) for b in webanno_blockwise(f.read())])

In [73]:
text[:100].split('\n')

['',
 '',
 'باب O',
 'ذكر O',
 'البقر O',
 '',
 'أنبأنا O',
 'عبد O',
 'الأول O',
 'بن O',
 'عيسى O',
 'قال O',
 'أنبأنا O',
 'عبد O',
 'الله O',
 'بن O',
 'محمد O',
 'ال']

In [16]:
def tsv_to_conll(in_dir, out_dir):
    tsv_basenames = (f for f in os.listdir(in_dir) if f.endswith('.tsv'))
    for tsv in tsv_basenames:
        tsv_fname = os.path.join(in_dir, tsv)
        conll_fname = os.path.join(out_dir, tsv[:-4] + '.conll')
        
        with open(tsv_fname, 'r') as f:
            tokens_labels = '\n\n'.join([wablock_to_conll(b, wa_to_BIO_labels)
                                         for b in webanno_blockwise(f.read())])

        with open(os.path.join(conll_fname), 'w') as out:
                    out.writelines(tokens_labels)

In [13]:
tsv_to_conll(folder_ner, folder_ner)

TypeError: __new__() takes 6 positional arguments but 7 were given

In [17]:
tsv_to_conll(folder_snd, folder_snd)

In [ ]:
def wa_to_df(wab:str):
    
    
    is_content = lambda l: l and not l.startswith('#')
    for line in wab.splitlines():
        is_content(line)
    

In [19]:
fname = os.path.join(folder_ner, '0597IbnJawzi.Mawducat.ShamAY0034138_3_ner.tsv')

In [25]:
splitfield = lambda f: [int(n) for n in f.split('-')]

In [50]:
df_snd = pd.read_csv(exe_snd, comment='#', sep='\t|-', 
                 names=('sent_no', 'tok_no', 'ch_ini', 'ch_end', 'token', 'label', 'empty')) 

/tmp/ipykernel_17960/3472671952.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_snd = pd.read_csv(exe_snd, comment='#', sep='\t|-',


In [51]:
df_snd[:20]

,sent_no,tok_no,ch_ini,ch_end,token,label,empty
0,1,1,1,4,باب,_,NaN
1,1,2,5,8,ذكر,_,NaN
2,1,3,9,14,البقر,_,NaN
3,1,4,14,15,.,_,NaN
4,2,1,16,20,1346,_,NaN
5,2,2,21,27,أنبأنا,snd[1],NaN
6,2,3,28,31,عبد,snd[1],NaN
7,2,4,32,37,الأول,snd[1],NaN
8,2,5,38,40,بن,snd[1],NaN
9,2,6,41,45,عيسى,snd[1],NaN


In [37]:
fname_snd = os.path.join(folder_snd, '0597IbnJawzi.Mawducat.ShamAY0034138_3_snd.tsv')

In [40]:
df_ner = pd.read_csv(exe_ner, comment='#', sep='\t|-', 
                 names=('sent_no', 'tok_no', 'ch_ini', 'ch_end', 'token', 'label_id', 'label')) 

/tmp/ipykernel_17960/2588397831.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ner = pd.read_csv(exe_ner, comment='#', sep='\t|-',


In [41]:
df_ner[:20]

,sent_no,tok_no,ch_ini,ch_end,token,label_id,label
0,1,1,1,4,باب,_,_
1,1,2,5,8,ذكر,_,_
2,1,3,9,14,البقر,_,_
3,1,4,14,15,.,_,_
4,2,1,16,20,1346,_,_
5,2,2,21,27,أنبأنا,_,_
6,2,3,28,31,عبد,*[1],PER[1]
7,2,4,32,37,الأول,*[1],PER[1]
8,2,5,38,40,بن,*[1],PER[1]
9,2,6,41,45,عيسى,*[1],PER[1]


In [52]:
df_snd.dropna(axis='columns', how='all')

,sent_no,tok_no,ch_ini,ch_end,token,label
0,1,1,1,4,باب,_
1,1,2,5,8,ذكر,_
2,1,3,9,14,البقر,_
3,1,4,14,15,.,_
4,2,1,16,20,1346,_
...,...,...,...,...,...,...
77644,1977,13,373823,373828,وصحبه,_
77645,1977,14,373829,373833,وسلم,_
77646,1977,15,373834,373840,تسليما,_
77647,1977,16,373841,373846,كثيرا,_


In [53]:
df_snd.label.str.split('[').str[0][:20]

0       _
1       _
2       _
3       _
4       _
5     snd
6     snd
7     snd
8     snd
9     snd
10    snd
11    snd
12    snd
13    snd
14    snd
15    snd
16    snd
17    snd
18    snd
19    snd
Name: label, dtype: object

In [153]:
# df.loc[:, f'BIO_{col}']
def bio_encode(df, col):
    """
    """

    # take 'snd[45]' or 'PER[3]' and keep only 'snd' resp. 'PER'
    res = df.loc[:, col].str.split('[').str[0]
    
    labels = set(res.unique()) - set('_')

    for label in labels:
        res = res.str.replace(label, f'I-{label}', regex=False)
          
    # deal separately with 'other'
    df.loc[:, f'BIO_{col}'] = res.str.replace('_', 'O').str.upper()
    
    for i, value in enumerate(df.loc[:, f'BIO_{col}']):
        if df.at[i, f'BIO_{col}'].startswith('I') and df.at[i-1, f'BIO_{col}'] == 'O':
            df.at[i, f'BIO_{col}'] = value.replace('I-', 'B-') 
    
    return df
    

In [154]:
organic_snd = bio_encode(df_snd, 'label')

In [155]:
organic_snd[:10]

,sent_no,tok_no,ch_ini,ch_end,token,label,empty,BIO_label
0,1,1,1,4,باب,_,NaN,O
1,1,2,5,8,ذكر,_,NaN,O
2,1,3,9,14,البقر,_,NaN,O
3,1,4,14,15,.,_,NaN,O
4,2,1,16,20,1346,_,NaN,O
5,2,2,21,27,أنبأنا,snd[1],NaN,B-SND
6,2,3,28,31,عبد,snd[1],NaN,I-SND
7,2,4,32,37,الأول,snd[1],NaN,I-SND
8,2,5,38,40,بن,snd[1],NaN,I-SND
9,2,6,41,45,عيسى,snd[1],NaN,I-SND


In [156]:
organic_ner = bio_encode(df_ner, 'label')

In [157]:
organic_ner[:10]

,sent_no,tok_no,ch_ini,ch_end,token,label_id,label,BIO_label
0,1,1,1,4,باب,_,_,O
1,1,2,5,8,ذكر,_,_,O
2,1,3,9,14,البقر,_,_,O
3,1,4,14,15,.,_,_,O
4,2,1,16,20,1346,_,_,O
5,2,2,21,27,أنبأنا,_,_,O
6,2,3,28,31,عبد,*[1],PER[1],B-PER
7,2,4,32,37,الأول,*[1],PER[1],I-PER
8,2,5,38,40,بن,*[1],PER[1],I-PER
9,2,6,41,45,عيسى,*[1],PER[1],I-PER


In [158]:
organic_snd

,sent_no,tok_no,ch_ini,ch_end,token,label,empty,BIO_label
0,1,1,1,4,باب,_,NaN,O
1,1,2,5,8,ذكر,_,NaN,O
2,1,3,9,14,البقر,_,NaN,O
3,1,4,14,15,.,_,NaN,O
4,2,1,16,20,1346,_,NaN,O
...,...,...,...,...,...,...,...,...
77644,1977,13,373823,373828,وصحبه,_,NaN,O
77645,1977,14,373829,373833,وسلم,_,NaN,O
77646,1977,15,373834,373840,تسليما,_,NaN,O
77647,1977,16,373841,373846,كثيرا,_,NaN,O


In [159]:
organic_ner

,sent_no,tok_no,ch_ini,ch_end,token,label_id,label,BIO_label
0,1,1,1,4,باب,_,_,O
1,1,2,5,8,ذكر,_,_,O
2,1,3,9,14,البقر,_,_,O
3,1,4,14,15,.,_,_,O
4,2,1,16,20,1346,_,_,O
...,...,...,...,...,...,...,...,...
77644,1977,13,373823,373828,وصحبه,_,_,O
77645,1977,14,373829,373833,وسلم,_,_,O
77646,1977,15,373834,373840,تسليما,_,_,O
77647,1977,16,373841,373846,كثيرا,_,_,O


In [157]:
prev = df_snd.label.shift(-1)
cur = df_snd.label

In [161]:
df_snd = df_snd.assign(start=(cur=='_') & (cur != prev))

In [163]:
df_snd[:46]

,sent_no,tok_no,ch_ini,ch_end,token,label,empty,BIO_label,change,start
0,1,1,1,4,باب,_,NaN,O,False,False
1,1,2,5,8,ذكر,_,NaN,O,False,False
2,1,3,9,14,البقر,_,NaN,O,False,False
3,1,4,14,15,.,_,NaN,O,False,False
4,2,1,16,20,1346,_,NaN,O,True,True
5,2,2,21,27,أنبأنا,snd[1],NaN,I-snd,False,False
6,2,3,28,31,عبد,snd[1],NaN,I-snd,False,False
7,2,4,32,37,الأول,snd[1],NaN,I-snd,False,False
8,2,5,38,40,بن,snd[1],NaN,I-snd,False,False
9,2,6,41,45,عيسى,snd[1],NaN,I-snd,False,False


In [164]:
trial = """
19-54	3633-3637	جابر	*[27]	PER[27]	
19-55	3638-3640	بن	*[27]	PER[27]	
19-56	3641-3645	سمرة	*[27]	PER[27]	
19-59	3654-3658	رسول	*[28]	PER[28]	
19-60	3659-3663	الله	*[28]	PER[28]	
"""

In [ ]:
for i, value in enumerate(df_clara.l_cl):
    if df_clara.at[i, 'l_cl'] == 'PER' and df_clara.at[i-1, 'l_cl'] == 'O':
        df_clara.at[i, 'l_cl'] = 'B-PERS'

In [ ]:
def merge_ner_snd(df_ner, df_snd):
    ner_snd = df_snd.join(df_ner['label_ner'])

In [ ]:
df.rename(columns={"A": "a", "B": "c"})

In [166]:
df_snd = df_snd.rename(columns={'BIO_label':'label_snd'})

In [164]:
df_ner = df_ner.rename(columns={'BIO_label':'label_ner'})

In [165]:
df_ner

,sent_no,tok_no,ch_ini,ch_end,token,label_id,label,label_ner
0,1,1,1,4,باب,_,_,O
1,1,2,5,8,ذكر,_,_,O
2,1,3,9,14,البقر,_,_,O
3,1,4,14,15,.,_,_,O
4,2,1,16,20,1346,_,_,O
...,...,...,...,...,...,...,...,...
77644,1977,13,373823,373828,وصحبه,_,_,O
77645,1977,14,373829,373833,وسلم,_,_,O
77646,1977,15,373834,373840,تسليما,_,_,O
77647,1977,16,373841,373846,كثيرا,_,_,O


In [178]:
ner_snd = pd.merge(df_snd, df_ner, on=['token'])[['label_snd', 'label_ner', 'token']]

In [179]:
ner_snd

,label_snd,label_ner,token
0,O,O,باب
1,O,O,باب
2,O,O,باب
3,O,O,باب
4,O,O,باب
...,...,...,...
90707204,O,O,تم
90707205,O,O,وآخرا
90707206,O,O,آله
90707207,O,O,وصحبه


In [180]:
ner_snd = pd.concat([df_snd, df_ner], axis=1)[['label_snd', 'label_ner', 'token']]

In [181]:
ner_snd

,label_snd,label_ner,token,token
0,O,O,باب,باب
1,O,O,ذكر,ذكر
2,O,O,البقر,البقر
3,O,O,.,.
4,O,O,1346,1346
...,...,...,...,...
77644,O,O,وصحبه,وصحبه
77645,O,O,وسلم,وسلم
77646,O,O,تسليما,تسليما
77647,O,O,كثيرا,كثيرا


In [ ]:
right.merge(left, on=['key1','key2'])[['A','B','C','key1','key2']]
result = pd.merge(left, right, on="key")

In [ ]:
df1 = df1. join(numbers)

In [189]:
ner_snd = df_snd.join(df_ner['label_ner'])[['label_snd', 'label_ner', 'token']]

In [190]:
ner_snd

,label_snd,label_ner,token
0,O,O,باب
1,O,O,ذكر
2,O,O,البقر
3,O,O,.
4,O,O,1346
...,...,...,...
77644,O,O,وصحبه
77645,O,O,وسلم
77646,O,O,تسليما
77647,O,O,كثيرا


In [186]:
ner_snd[['label_snd', 'label_ner', 'token']]

,label_snd,label_ner,token
0,O,O,باب
1,O,O,ذكر
2,O,O,البقر
3,O,O,.
4,O,O,1346
...,...,...,...
77644,O,O,وصحبه
77645,O,O,وسلم
77646,O,O,تسليما
77647,O,O,كثيرا


In [188]:
ner_snd[['label_snd', 'label_ner', 'token']].to_csv(os.path.join(folder_ner, 'trial.conll'),
                                                    index=False, header=None, sep=' ')

In [194]:
def webanno_to_df(fpath:str, col_names:tuple):
    df = pd.read_csv(fpath, comment='#', sep='\t|-', names=col_names) 
    
    return bio_encode(df, 'label')

In [195]:
ner_col_names = ('sent_no', 'tok_no', 'ch_ini', 'ch_end', 'token', 'label_id', 'label')
snd_col_names = ('sent_no', 'tok_no', 'ch_ini', 'ch_end', 'token', 'label', 'empty')

In [196]:
def bio_encode(df, col):
    """
    """

    # take 'snd[45]' or 'PER[3]' and keep only 'snd' resp. 'PER'
    res = df.loc[:, col].str.split('[').str[0]
    
    labels = set(res.unique()) - set('_')

    for label in labels:
        res = res.str.replace(label, f'I-{label}', regex=False)
          
    # deal separately with 'other'
    df.loc[:, f'BIO_{col}'] = res.str.replace('_', 'O').str.upper()
    
    for i, value in enumerate(df.loc[:, f'BIO_{col}']):
        if df.at[i, f'BIO_{col}'].startswith('I') and df.at[i-1, f'BIO_{col}'] == 'O':
            df.at[i, f'BIO_{col}'] = value.replace('I-', 'B-') 
    
    return df

In [197]:
def merge_ner_snd(fpath_ner, fpath_snd, fpath_mix):
    
    df_ner = webanno_to_df(fpath_ner, ner_col_names)
    df_snd = webanno_to_df(fpath_snd, snd_col_names)
    
    df_snd = df_snd.rename(columns={'BIO_label':'label_snd'})
    df_ner = df_ner.rename(columns={'BIO_label':'label_ner'})
    
    merged = df_snd.join(df_ner['label_ner'])[['label_snd', 'label_ner', 'token']]
    
    return merged.to_csv(fpath_mix, index=False, header=None, sep=' ')

In [ ]:
def webanno_to_conll(folder:str):
    
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in files:
            
    

In [193]:
for root, dirs, files in os.walk(f'{home}/nlp/texts/tsv/', topdown=True):
    
    for idx, name in enumerate(files):
        print(name)

0597IbnJawzi.Mawducat.ShamAY0034138_2_ner.tsv
0597IbnJawzi.Mawducat.ShamAY0034138_1_snd.tsv
0597IbnJawzi.Mawducat.ShamAY0034138_3_ner.tsv
0597IbnJawzi.Mawducat.ShamAY0034138_1_ner.tsv
0597IbnJawzi.Mawducat.ShamAY0034138_3_snd.tsv
0597IbnJawzi.Mawducat.ShamAY0034138_2_snd.tsv


In [198]:
ner_1 = '0597IbnJawzi.Mawducat.ShamAY0034138_1_ner.tsv'
ner_2 = '0597IbnJawzi.Mawducat.ShamAY0034138_2_ner.tsv'
ner_3 = '0597IbnJawzi.Mawducat.ShamAY0034138_3_ner.tsv'

snd_1 = '0597IbnJawzi.Mawducat.ShamAY0034138_1_snd.tsv'
snd_2 = '0597IbnJawzi.Mawducat.ShamAY0034138_2_snd.tsv'
snd_3 = '0597IbnJawzi.Mawducat.ShamAY0034138_3_snd.tsv'

In [199]:
folder = f'{home}/nlp/texts/tsv/'

In [203]:
tu = ((ner_1, snd_1), (ner_2, snd_2), (ner_3, snd_3))

In [204]:
tu

(('0597IbnJawzi.Mawducat.ShamAY0034138_1_ner.tsv',
  '0597IbnJawzi.Mawducat.ShamAY0034138_1_snd.tsv'),
 ('0597IbnJawzi.Mawducat.ShamAY0034138_2_ner.tsv',
  '0597IbnJawzi.Mawducat.ShamAY0034138_2_snd.tsv'),
 ('0597IbnJawzi.Mawducat.ShamAY0034138_3_ner.tsv',
  '0597IbnJawzi.Mawducat.ShamAY0034138_3_snd.tsv'))

In [208]:
for fname in tu:
    print(fname[1][:-4])

0597IbnJawzi.Mawducat.ShamAY0034138_1_snd.
0597IbnJawzi.Mawducat.ShamAY0034138_2_snd.
0597IbnJawzi.Mawducat.ShamAY0034138_3_snd.


In [209]:
for fname in tu:
    ner = os.path.join(folder, fname[0])
    snd = os.path.join(folder, fname[1])
    merge_ner_snd(ner, snd, f'{fname[0][:-4]}-snd.conll')

/tmp/ipykernel_17960/1783671721.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(fpath, comment='#', sep='\t|-', names=col_names)
